In [89]:
!pip install statistics

In [90]:
!pip install astropy

In [91]:
# Converting DZStats by Kurt Sundell into a Python program

In [92]:
# Kolmogorov-Smirnov Test Code
import numpy as np
from scipy import stats

df = pd.read_excel('ForAgeCalc.xlsx')
data1 = df['RNHW'].dropna().to_numpy()
data2 = df['Gilder GS-11'].dropna().to_numpy()

stats.ks_2samp(data1, data2, nan_policy='omit')

KstestResult(statistic=np.float64(0.1524876305662452), pvalue=np.float64(0.10830244926279575), statistic_location=np.float64(1218.0), statistic_sign=np.int8(1))

In [93]:
df = pd.read_excel('ForAgeCalc.xlsx')
data3 = df['PB'].to_numpy()
data4 = df['FL3'].to_numpy()

stats.ks_2samp(data3, data4, alternative='two-sided', nan_policy='omit')

KstestResult(statistic=np.float64(0.06886694386694386), pvalue=np.float64(0.9092391643909453), statistic_location=np.float64(1093.01), statistic_sign=np.int8(-1))

In [94]:
df = pd.read_excel('ForAgeCalc.xlsx')
data5 = df['E79'].to_numpy()
data6 = df['NY14-TAP-22'].to_numpy()

stats.ks_2samp(data5, data6, alternative='two-sided', nan_policy='omit')

KstestResult(statistic=np.float64(0.4356299212598425), pvalue=np.float64(4.356451974239638e-11), statistic_location=np.float64(1000.0), statistic_sign=np.int8(1))

In [95]:
# Kolmogorov-Smirnov Test Function
def ks_test(file_name, col_name1, col_name2):
    df = pd.read_excel(file_name)
    data1 = df[col_name1].to_numpy()
    data2 = df[col_name2].to_numpy()
    return stats.ks_2samp(data1, data2, alternative='two-sided', nan_policy='omit')

In [96]:
ks_test('ForAgeCalc.xlsx', 'PB', 'FL3')

KstestResult(statistic=np.float64(0.06886694386694386), pvalue=np.float64(0.9092391643909453), statistic_location=np.float64(1093.01), statistic_sign=np.int8(-1))

In [97]:
# Kuiper Test Code
import astropy
from astropy import stats
from astropy.stats import kuiper_two

df = pd.read_excel('ForAgeCalc.xlsx')
data7 = df['RNHE'].dropna().to_numpy()
data8 = df['RNHW'].dropna().to_numpy()
kuiper_two(data7, data8)

(0.2797118847539016, np.float64(0.003042951016084474))

In [98]:
print(len(data8))

136


In [99]:
# Kuiper Test Function
def kuiper_test(file_name, col_name1, col_name2):
    df = pd.read_excel(file_name)
    data1 = df[col_name1].dropna().to_numpy()
    data2 = df[col_name2].dropna().to_numpy()
    return kuiper_two(data1, data2)

In [100]:
kuiper_test('ForAgeCalc.xlsx', 'PB', 'FL3')

(0.1288981288981289, np.float64(0.7596953736522171))

In [101]:
kuiper_test('ForAgeCalc.xlsx', 'RNHW', 'Gilder GS-11')

(0.17255360087960417, np.float64(0.29292275294596803))

In [102]:
kuiper_test('ForAgeCalc.xlsx', 'E79', 'NY14-TAP-22')

(0.4439632545931735, np.float64(1.127755151380478e-09))

In [103]:
import numpy as np
from numpy import copy, sort, amax, arange, exp, sqrt, abs, floor, searchsorted
import itertools

def kuiper_FPP(D,N):
    """Compute the false positive probability for the Kuiper statistic.

    Uses the set of four formulas described in Paltani 2004; they report 
    the resulting function never underestimates the false positive probability 
    but can be a bit high in the N=40..50 range. (They quote a factor 1.5 at 
    the 1e-7 level.

    Parameters
    ----------
    D : float
        The Kuiper test score.
    N : float
        The effective sample size.

    Returns
    -------
    fpp : float
        The probability of a score this large arising from the null hypothesis.

    Reference
    ---------
    Paltani, S., "Searching for periods in X-ray observations using 
    Kuiper's test. Application to the ROSAT PSPC archive", Astronomy and
    Astrophysics, v.240, p.789-790, 2004.

    """
    if D<0. or D>2.:
        raise ValueError("Must have 0<=D<=2 by definition of the Kuiper test")

    if D<2./N:
        return 1. - factorial(N)*(D-1./N)**(N-1)
    elif D<3./N:
        k = -(N*D-1.)/2.
        r = sqrt(k**2 - (N*D-2.)/2.)
        a, b = -k+r, -k-r
        return 1. - factorial(N-1)*(b**(N-1.)*(1.-a)-a**(N-1.)*(1.-b))/float(N)**(N-2)*(b-a)
    elif (D>0.5 and N%2==0) or (D>(N-1.)/(2.*N) and N%2==1):
        def T(t):
            y = D+t/float(N)
            return y**(t-3)*(y**3*N-y**2*t*(3.-2./N)/N-t*(t-1)*(t-2)/float(N)**2)
        s = 0.
        # NOTE: the upper limit of this sum is taken from Stephens 1965
        for t in xrange(int(floor(N*(1-D)))+1):
            term = T(t)*comb(N,t)*(1-D-t/float(N))**(N-t-1)
            s += term
        return s
    else:
        z = D*sqrt(N) 
        S1 = 0.
        term_eps = 1e-12
        abs_eps = 1e-100
        for m in itertools.count(1):
            T1 = 2.*(4.*m**2*z**2-1.)*exp(-2.*m**2*z**2)
            so = S1
            S1 += T1
            if abs(S1-so)/(abs(S1)+abs(so))<term_eps or abs(S1-so)<abs_eps:
                break
        S2 = 0.
        for m in itertools.count(1):
            T2 = m**2*(4.*m**2*z**2-3.)*exp(-2*m**2*z**2)
            so = S2
            S2 += T2
            if abs(S2-so)/(abs(S2)+abs(so))<term_eps or abs(S1-so)<abs_eps:
                break
        return S1 - 8*D/(3.*sqrt(N))*S2

In [108]:
def kuiper_dos(data1, data2):
    """Compute the Kuiper statistic to compare two samples.

    Parameters
    ----------
    data1 : array-like
        The first set of data values.
    data2 : array-like
        The second set of data values.
    
    Returns
    -------
    D : float
        The raw test statistic.
    fpp : float
        The probability of obtaining two samples this different from
        the same distribution.

    Notes
    -----
    Warning: the fpp is quite approximate, especially for small samples.

    """
    data1, data2 = sort(data1), sort(data2)

    if len(data2)<len(data1):
        data1, data2 = data2, data1

    cdfv1 = searchsorted(data2, data1)/float(len(data2)) # this could be more efficient
    cdfv2 = searchsorted(data1, data2)/float(len(data1)) # this could be more efficient
    D = (amax(cdfv1-arange(len(data1))/float(len(data1))) + 
            amax(cdfv2-arange(len(data2))/float(len(data2))))

    Ne = len(data1)*len(data2)/float(len(data1)+len(data2))
    return D, kuiper_FPP(D, Ne)

In [114]:
df = pd.read_excel('ForAgeCalc.xlsx')
data_one = df['RNHW'].dropna().to_numpy()
data_two = df['RNHE'].dropna().to_numpy()

In [113]:
kuiper_dos(data_one, data_two)

(np.float64(0.2797118847539015), np.float64(0.00433500859876872))